In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils
import utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_filter_map_plot

E0415 22:14:21.533650042    1355 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0415 22:14:24.121443542    1355 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies


In [2]:
## parameters cell
itp_id = 290

In [3]:
airtable_organizations = (
    tbl.airtable.california_transit_organizations()
    >> select(_.itp_id, _.name, _.caltrans_district,
              _.website, _.ntp_id, _.drmt_organization_name)
    >> filter(_.itp_id == itp_id)
    >> collect()
)

In [4]:
# airtable_organizations

In [5]:
analysis_date = dt.date(2022, 3, 30)

In [6]:
trips = pd.read_parquet(f'{utils.GCS_FILE_PATH}rt_trips/{itp_id}_03_30.parquet')

In [7]:
stop_delay = gpd.read_parquet(f'{utils.GCS_FILE_PATH}stop_delay_views/{itp_id}_03_30.parquet')
stop_delay['arrival_time'] = stop_delay.arrival_time.map(lambda x: dt.datetime.fromisoformat(x))
stop_delay['actual_time'] = stop_delay.actual_time.map(lambda x: dt.datetime.fromisoformat(x))

In [8]:
%%capture
routelines = utils.get_routelines(itp_id, analysis_date)

In [9]:
%%capture
rt_day = rt_filter_map_plot.RtFilterMapper(trips, stop_delay, routelines)

In [10]:
%%capture
rt_day.set_filter(start_time='15:00', end_time='19:00')

In [11]:
rt_day.describe_delayed_routes()

PM Peak most delayed routes: 
 * 112, Linda Mar Park & Ride - Colma BART, Southbound: 24.0 minutes late on average
 * 17, Linda Mar Park & Ride - Pescadero, Northbound: 19.0 minutes late on average
 * 18, Main/7th - HMBHS - Moonridge Apts, Northbound: 14.0 minutes late on average
 * 28, Serramonte Center - South City High, Westbound: 13.0 minutes late on average
 * 110, Daly City BART - Linda Mar Park & Ride, Southbound: 10.0 minutes late on average

In [2]:
import gcsfs
fs = gcsfs.GCSFileSystem()

In [3]:
fs_list = fs.ls(f'{utils.GCS_FILE_PATH}rt_trips/')

In [4]:
ran_operators = [int(path.split('rt_trips/')[1].split('_')[0])
                 for path in fs_list
                 if path.split('rt_trips/')[1]]

In [5]:
ran_operators

[110,
 126,
 127,
 135,
 148,
 159,
 167,
 170,
 182,
 188,
 194,
 208,
 218,
 221,
 226,
 235,
 243,
 246,
 259,
 269,
 273,
 278,
 282,
 284,
 290,
 293,
 294,
 295,
 300,
 301,
 30,
 310,
 315,
 336,
 349,
 350,
 361,
 45,
 4,
 75]

In [6]:
import os

In [7]:
nb_exists = os.listdir('../docs/rt_delay/')

In [8]:
nb_exists = [int(nb.split('id_')[1].split('.')[0]) for nb in nb_exists if 'id_' in nb]

In [9]:
[operator for operator in ran_operators if operator not in nb_exists]

[273]

In [18]:
import folium

In [25]:
gdf = rt_day.stop_segment_speed_view.to_crs('EPSG:4326')
gdf = gdf.round({'avg_mph': 1, '_20p_mph': 1, 'shape_miles': 1,
                'trips_per_hour': 1}) ##round for display
gdf = gdf >> select(-_.service_date, -_.last_loc, -_.shape_meters,
               -_.meters_from_last, -_.speed_mph, -_.speed_from_last,
               -_.trip_id, -_.trip_key, -_.n_trips) ## drop unused cols for smaller map size

In [26]:
test = folium.GeoJson(
    gdf,
)

In [ ]:
test?

In [ ]:
# test.data

In [29]:
def reduce_geojson_precision(geojson, decimal_places = 4):
    '''
    Rounds coordinates in geojson to specified number of decimal places. 
    Useful for shrinking map sizes while maintaining accuracy.
    '''
    features = geojson['features']
    for feature in features:
        coords = feature['geometry']['coordinates']
        if len(coords) == 1:
            coords = coords[0]
        try:
            coords = [[round(x[0], decimal_places), round(x[1], decimal_places)] for x in coords]
            feature['geometry']['coordinates'] = coords
            feature['bbox'] =[round(x, decimal_places) for x in feature['bbox']]
        except:
            # print(coords)
            pass
    return geojson

In [34]:
test.data['features']

[{'id': '0',
  'type': 'Feature',
  'properties': {'_20p_mph': 1.9,
   'avg_mph': 8.1,
   'delay_chg_sec': 94.0,
   'delay_seconds': 94,
   'direction_id': '1',
   'route_id': '3481',
   'route_short_name': '3',
   'seconds_from_last': 745.0,
   'shape_id': '25888',
   'stop_id': '50',
   'stop_name': '4TH SB & WILSHIRE FS',
   'stop_sequence': 2,
   'trips_per_hour': 2.0},
  'geometry': {'type': 'LineString',
   'coordinates': [[-118.4959, 34.0185],
    [-118.4958, 34.0185],
    [-118.4953, 34.019],
    [-118.4949, 34.0193],
    [-118.496, 34.0203],
    [-118.4965, 34.0206],
    [-118.4969, 34.0203],
    [-118.4974, 34.0199],
    [-118.4984, 34.0191],
    [-118.498, 34.0188]]},
  'bbox': [-118.4984, 34.0185, -118.4949, 34.0206]},
 {'id': '1',
  'type': 'Feature',
  'properties': {'_20p_mph': 6.2,
   'avg_mph': 11.3,
   'delay_chg_sec': 13.0,
   'delay_seconds': 107,
   'direction_id': '1',
   'route_id': '3481',
   'route_short_name': '3',
   'seconds_from_last': 61.0,
   'shape_id': 

In [35]:
[feature for feature in test.data['features'] if len(feature['geometry']['coordinates']) == 1]

[]

In [43]:
import json

In [44]:
json.loads(gdf.to_json())

{'type': 'FeatureCollection',
 'features': [{'id': '0',
   'type': 'Feature',
   'properties': {'_20p_mph': 1.9,
    'avg_mph': 8.1,
    'delay_chg_sec': 94.0,
    'delay_seconds': 94,
    'direction_id': '1',
    'route_id': '3481',
    'route_short_name': '3',
    'seconds_from_last': 745.0,
    'shape_id': '25888',
    'stop_id': '50',
    'stop_name': '4TH SB & WILSHIRE FS',
    'stop_sequence': 2,
    'trips_per_hour': 2.0},
   'geometry': {'type': 'LineString',
    'coordinates': [[-118.49591147767028, 34.01846758062823],
     [-118.49582, 34.01853999999998],
     [-118.49531000000002, 34.01895900000002],
     [-118.494889, 34.01929899999998],
     [-118.49604900000001, 34.020289000000005],
     [-118.496459, 34.02062000000003],
     [-118.49689999999998, 34.02025999999995],
     [-118.49738, 34.01985899999999],
     [-118.49835900000001, 34.01908],
     [-118.49797022587795, 34.018763951861786]]}},
  {'id': '1',
   'type': 'Feature',
   'properties': {'_20p_mph': 6.2,
    'avg_m

In [46]:
data1 = reduce_geojson_precision(geojson=json.loads(gdf.to_json()))

In [47]:
m = folium.Map()